# RAG Evaluation

## Load Dependencies

In [1]:
%pip install azure-ai-evaluation
%pip install promptflow-azure

  Using cached protobuf-5.29.4-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
  Using cached opentelemetry_api-1.31.1-py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 22.7 MB/s eta 0:00:00
Using cached opentelemetry_api-1.31.1-py3-none-any.whl (65 kB)
Using cached protobuf-5.29.4-cp38-abi3-manylinux2014_x86_64.whl (319 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
  Attempting uninstall: pillow
    Found existing installation: pillow 11.1.0
    Uninstalling pillow-11.1.0:
      Successfully uninstalled pillow-11.1.0
  Attempting uninstall: opentelemetry-api
    Found existing installation: opentelemetry-api 1.27.0
    Uninstalling opentelemetry-api-1.27.0:
      Successfully uninstalled opentelemetry-api-1.27.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. T

## Load Azure configurations

You always need to run this!

In [1]:
from dotenv import load_dotenv
import os

load_dotenv() # take environment variables from .env.

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

model_config = {
    "azure_endpoint": azure_openai_endpoint,
    "api_key": azure_openai_key,
    "azure_deployment": azure_openai_deployment,
}

azure_subscription_id = os.getenv("AZURE_SUBSCRIPTION_ID")
azure_resource_group_name = os.getenv("AZURE_RESOURCE_GROUP_NAME")
azure_project_name = os.getenv("AZURE_PROJECT_NAME")

azure_ai_project = {
    "subscription_id": azure_subscription_id,
    "resource_group_name": azure_resource_group_name,
    "project_name": azure_project_name,
}

## Get the first row to test

In [2]:
import json

# Load JSON data from a file
with open('../Data/output/nasaeval.jsonl', 'r') as file:
    data = [json.loads(line) for line in file]

# Assuming the JSON structure is a list of dictionaries and we want the first row
first_row = data[0]

# Assign values to variables
context = first_row['context']
query = first_row['query']
ground_truth = first_row['ground_truth']
response = first_row['response']

## Performance Evaluators

In [4]:
from azure.ai.evaluation import GroundednessEvaluator, RelevanceEvaluator, CoherenceEvaluator, FluencyEvaluator, SimilarityEvaluator, F1ScoreEvaluator
from azure.ai.evaluation import RougeScoreEvaluator, RougeType
from azure.ai.evaluation import BleuScoreEvaluator
from azure.ai.evaluation import MeteorScoreEvaluator
from azure.ai.evaluation import GleuScoreEvaluator

groundedness_eval = GroundednessEvaluator(model_config)
groundedness_score = groundedness_eval(
    response=response,
    context=context,
)

relevance_eval = RelevanceEvaluator(model_config)
relevance_score = relevance_eval(
    response=response,
    context=context,
    query=query
)

coherence_eval = CoherenceEvaluator(model_config)
coherence_score = coherence_eval(
    response=response,
    query=query
)

fluency_eval = FluencyEvaluator(model_config)
fluency_score = fluency_eval(
    response=response,
    query=query
)

similarity_eval = SimilarityEvaluator(model_config)
similarity_score = similarity_eval(
    response=response,
    query=query,
    ground_truth=ground_truth
)

f1_eval = F1ScoreEvaluator()
f1_score = f1_eval(
    response=response,
    ground_truth=ground_truth
)

# There are several types of ROUGE metrics: ROUGE_1, ROUGE_2, ROUGE_3, ROUGE_4, ROUGE_5, and ROUGE_L.
rouge_eval = RougeScoreEvaluator(rouge_type=RougeType.ROUGE_1)
rouge_score = rouge_eval(
    response=response,
    ground_truth=ground_truth,
)

bleu_eval = BleuScoreEvaluator()
bleu_score = bleu_eval(
    response=response,
    ground_truth=ground_truth
)

meteor_eval = MeteorScoreEvaluator(
    alpha=0.9,
    beta=3.0,
    gamma=0.5
)
meteor_score = meteor_eval(
    response=response,
    ground_truth=ground_truth,
)

gleu_eval = GleuScoreEvaluator()
gleu_score = gleu_eval(
    response=response,
    ground_truth=ground_truth,
)

print(groundedness_score)
print(relevance_score)
print(coherence_score)
print(fluency_score)
print(similarity_score)
print(f1_score)
print(rouge_score)
print(bleu_score)
print(meteor_score)
print(gleu_score)

{'groundedness': 5.0, 'gpt_groundedness': 5.0, 'groundedness_reason': 'The RESPONSE accurately and thoroughly conveys all essential information from the CONTEXT without introducing unsupported details or omitting critical points.'}
{'relevance': 5.0, 'gpt_relevance': 5.0, 'relevance_reason': 'The response fully addresses the query with accurate and complete information, including additional relevant insights about the formation and arrangement of clouds.'}
{'coherence': 4.0, 'gpt_coherence': 4.0, 'coherence_reason': 'The response is coherent and effectively addresses the question with a logical sequence of ideas and clear connections between sentences. It provides detailed information about the satellite image and the phenomena observed, making it easy to understand.'}
{'fluency': 4.0, 'gpt_fluency': 4.0, 'fluency_reason': 'The response is well-articulated, with good control of grammar and a varied vocabulary. The sentences are complex and well-structured, demonstrating coherence and c

## Risk and Safety Metrics

1. Install Azure CLI in Github Codespaces
- curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash

2. Login with your Azure account 
- az login --use-device-code
- Once you've logged in, select your subscription in the terminal.

In [3]:
from azure.ai.evaluation import ViolenceEvaluator, HateUnfairnessEvaluator, SelfHarmEvaluator,SexualEvaluator
from azure.identity import DefaultAzureCredential

violence_eval = ViolenceEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
violence_score = violence_eval(query=query, response=response)

hateunfairness_eval = HateUnfairnessEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
hateunfairness_score = hateunfairness_eval(query=query, response=response)

selfharm_eval = SelfHarmEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
selfharm_score = selfharm_eval(query=query, response=response)

sexual_eval = SexualEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
sexual_score = sexual_eval(query=query, response=response)

print(violence_score)
print(hateunfairness_score)
print(selfharm_score)
print(sexual_score)

Class ViolenceEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class HateUnfairnessEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SelfHarmEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SexualEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


{'violence': 'Very low', 'violence_score': 0, 'violence_reason': "The system's response provides an informative and educational description of the Aqua satellite image of the Amazon rainforest. It discusses the formation of cumulus clouds and their arrangement without any mention of violence or physical harm. The content is purely scientific and does not contain any violent language or imagery."}
{'hate_unfairness': 'Very low', 'hate_unfairness_score': 0, 'hate_unfairness_reason': "The system's response provides a factual description of the Aqua satellite image of the Amazon rainforest in Brazil and Bolivia. It includes details about the formation of cumulus clouds and their arrangement, without any mention of social groups or ungrounded inference of sensitive attributes. There is no negative sentiment or unfair language present in the response."}
{'self_harm': 'Very low', 'self_harm_score': 0, 'self_harm_reason': "The system's response provides a detailed description of the Aqua satel

## Evaluate test dataset with the Performance Evaluators and Risk and Safety Metrics

In [7]:
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import GroundednessEvaluator, RetrievalEvaluator, RelevanceEvaluator, CoherenceEvaluator, FluencyEvaluator, SimilarityEvaluator, F1ScoreEvaluator
from azure.ai.evaluation import RougeScoreEvaluator, RougeType
from azure.ai.evaluation import BleuScoreEvaluator
from azure.ai.evaluation import MeteorScoreEvaluator
from azure.ai.evaluation import GleuScoreEvaluator
from azure.ai.evaluation import ViolenceEvaluator, HateUnfairnessEvaluator, SelfHarmEvaluator,SexualEvaluator
from azure.identity import DefaultAzureCredential
import pandas as pd

groundedness_eval = GroundednessEvaluator(model_config)
retrieval_eval = RetrievalEvaluator(model_config)
relevance_eval = RelevanceEvaluator(model_config)
coherence_eval = CoherenceEvaluator(model_config)
fluency_eval = FluencyEvaluator(model_config)
similarity_eval = SimilarityEvaluator(model_config)
f1_eval = F1ScoreEvaluator()
rouge_eval = RougeScoreEvaluator(rouge_type=RougeType.ROUGE_1)
bleu_eval = BleuScoreEvaluator()
meteor_eval = MeteorScoreEvaluator(
    alpha=0.9,
    beta=3.0,
    gamma=0.5
)
gleu_eval = GleuScoreEvaluator()
violence_eval = ViolenceEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
hateunfairness_eval = HateUnfairnessEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
selfharm_eval = SelfHarmEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
sexual_eval = SexualEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())

path = "../Data/output/nasaeval.jsonl"

result = evaluate(
    data=path, # provide your data here
    evaluators={
        "groundedness": groundedness_eval,
        "retrieval": retrieval_eval,
        "relevance": relevance_eval,
        "coherence": coherence_eval,
        "fluency": fluency_eval,
        "similarity":similarity_eval,
        "f1_score": f1_eval,
        "rouge_score": rouge_eval,
        "bleu_score": bleu_eval,
        "meteor_score": meteor_eval,
        "gleu_score": gleu_eval,
        "violence_score": violence_eval,
        "hateunfairness_score": hateunfairness_eval,
        "selfharm_score": selfharm_eval,
        "sexual_score": sexual_eval         
    },
    # column mapping
    evaluator_config={
        "default": {
            "query": "${data.query}",
            "response": "${data.response}",
            "context": "${data.context}",
            "ground_truth": "${data.ground_truth}"
        }
    }
)

df = pd.DataFrame(result["rows"])
# Save the DataFrame to a CSV file
df.to_csv('../Data/output/nasaevalresult.csv', index=False)

print("DataFrame has been successfully saved to nasaevalresult.csv")

[2025-04-18 01:33:22 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-04-18 01:33:22 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_u43nm_a9_20250418_013322_849410, log path: /home/codespace/.promptflow/.runs/azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_u43nm_a9_20250418_013322_849410/logs.txt
[2025-04-18 01:33:22 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-04-18 01:33:22 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-04-18 01:33:22 +0000][promptflow._core.entry_meta_ge

Prompt flow service has started...
Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_similarity_similarity_asyncsimilarityevaluator_ojbmpb35_20250418_013322_853086
Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_u43nm_a9_20250418_013322_849410
Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_nn9alfn1_20250418_013322_854248
Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_yoynti_t_20250418_013322_854348
Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:

[2025-04-18 01:33:25 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-04-18 01:33:25 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_f1_score_f1_score_asyncf1scoreevaluator_a6oz1pur_20250418_013325_176196, log path: /home/codespace/.promptflow/.runs/azure_ai_evaluation_evaluators_f1_score_f1_score_asyncf1scoreevaluator_a6oz1pur_20250418_013325_176196/logs.txt


Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_f1_score_f1_score_asyncf1scoreevaluator_a6oz1pur_20250418_013325_176196


[2025-04-18 01:33:26 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-04-18 01:33:26 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_rouge_rouge_asyncrougescoreevaluator_a7mr6ox4_20250418_013326_288624, log path: /home/codespace/.promptflow/.runs/azure_ai_evaluation_evaluators_rouge_rouge_asyncrougescoreevaluator_a7mr6ox4_20250418_013326_288624/logs.txt


...

Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_1a2sc9ue_20250418_013327_634627


[2025-04-18 01:33:27 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-04-18 01:33:27 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_1a2sc9ue_20250418_013327_634627, log path: /home/codespace/.promptflow/.runs/azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_1a2sc9ue_20250418_013327_634627/logs.txt


Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_j6g_svhy_20250418_013328_627307


[2025-04-18 01:33:28 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-04-18 01:33:28 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_j6g_svhy_20250418_013328_627307, log path: /home/codespace/.promptflow/.runs/azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_j6g_svhy_20250418_013328_627307/logs.txt


2025-04-18 01:34:03 +0000   10810 execution.bulk     INFO     Finished 6 / 8 lines.
2025-04-18 01:34:03 +0000   10810 execution.bulk     INFO     Average execution time for completed lines: 5.78 seconds. Estimated time for incomplete lines: 11.56 seconds.
2025-04-18 01:34:19 +0000   10810 execution.bulk     INFO     Finished 6 / 8 lines.
2025-04-18 01:34:19 +0000   10810 execution.bulk     INFO     Average execution time for completed lines: 8.55 seconds. Estimated time for incomplete lines: 17.1 seconds.
2025-04-18 01:34:19 +0000   10810 execution.bulk     INFO     Finished 7 / 8 lines.
2025-04-18 01:34:19 +0000   10810 execution.bulk     INFO     Average execution time for completed lines: 7.37 seconds. Estimated time for incomplete lines: 7.37 seconds.
2025-04-18 01:34:19 +0000   10810 execution.bulk     INFO     Finished 7 / 8 lines.
2025-04-18 01:34:19 +0000   10810 execution.bulk     INFO     Average execution time for completed lines: 7.25 seconds. Estimated time for incomplete 

## Assign yourself the Proper role to Track results in Azure AI Foundry

1. Get your user ID

az ad signed-in-user show --query id --output tsv

2. Assign yourself the Storage Blob Data Contributor role in the Resource Group where the Azure AI Foundry project is. Replace the placeholder text with your subscription ID, resource group, and user ID.

az role assignment create --role "Storage Blob Data Contributor" --scope /subscriptions/mySubscriptionID/resourceGroups/myResourceGroupName --assignee-principal-type User --assignee-object-id "user-id"

Example: az role assignment create --role "Storage Blob Data Contributor" --scope /subscriptions/f08cda90-375b-4b3e-a105-4656379a94ab/reso
urceGroups/rg-Ziggy-ForEvaluation-AzureAIFoundry --assignee-principal-type User --assignee-object-id effb07cd-dc40-4b91-a120-32464c95a844



## Run Evaluation and Track in Azure AI Foundry

In [ ]:
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import GroundednessEvaluator, RetrievalEvaluator, RelevanceEvaluator, CoherenceEvaluator, FluencyEvaluator, SimilarityEvaluator, F1ScoreEvaluator
from azure.ai.evaluation import RougeScoreEvaluator, RougeType
from azure.ai.evaluation import BleuScoreEvaluator
from azure.ai.evaluation import MeteorScoreEvaluator
from azure.ai.evaluation import GleuScoreEvaluator
from azure.ai.evaluation import ViolenceEvaluator, HateUnfairnessEvaluator, SelfHarmEvaluator,SexualEvaluator
from azure.identity import DefaultAzureCredential
import pandas as pd

groundedness_eval = GroundednessEvaluator(model_config)
retrieval_eval = RetrievalEvaluator(model_config)
relevance_eval = RelevanceEvaluator(model_config)
coherence_eval = CoherenceEvaluator(model_config)
fluency_eval = FluencyEvaluator(model_config)
similarity_eval = SimilarityEvaluator(model_config)
f1_eval = F1ScoreEvaluator()
rouge_eval = RougeScoreEvaluator(rouge_type=RougeType.ROUGE_1)
bleu_eval = BleuScoreEvaluator()
meteor_eval = MeteorScoreEvaluator(
    alpha=0.9,
    beta=3.0,
    gamma=0.5
)
gleu_eval = GleuScoreEvaluator()
violence_eval = ViolenceEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
hateunfairness_eval = HateUnfairnessEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
selfharm_eval = SelfHarmEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
sexual_eval = SexualEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())

path = "../Data/output/nasaeval.jsonl"

result = evaluate(
    data=path, # provide your data here
    evaluators={
        "groundedness": groundedness_eval,
        "retrieval": retrieval_eval,
        "relevance": relevance_eval,
        "coherence": coherence_eval,
        "fluency": fluency_eval,
        "similarity": similarity_eval,
        "f1_score": f1_eval,
        "rouge_score": rouge_eval,
        "bleu_score": bleu_eval,
        "meteor_score": meteor_eval,
        "gleu_score": gleu_eval,
        "violence_score": violence_eval,
        "hateunfairness_score": hateunfairness_eval,
        "selfharm_score": selfharm_eval,
        "sexual_score": sexual_eval 
    },
    # column mapping
    evaluator_config={
        "default": {
            "query": "${data.query}",
            "response": "${data.response}",
            "context": "${data.context}",
            "ground_truth": "${data.ground_truth}"
        }
    },
    azure_ai_project = azure_ai_project
)


Class ViolenceEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class HateUnfairnessEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SelfHarmEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SexualEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
[2025-04-18 02:02:12 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-04-18 02:02:12 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_303qb3l8_20250418_020212_44

Prompt flow service has started...
Prompt flow service has started...
Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_bxo1bdez_20250418_020212_444871
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_57hta4b0_20250418_020212_434082
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_303qb3l8_20250418_020212_445514
Prompt flow service has started...
Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_similarity_similarity_asyncsimilarityevaluator_n6w4gu2y_20250418_020212_441231
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai

[2025-04-18 02:02:14 +0000][promptflow][WARNING] - Found existing /tmp/azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase__ut4jyi3/flow.flex.yaml, will not respect it in runtime.
[2025-04-18 02:02:14 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-04-18 02:02:14 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_f1_score_f1_score_asyncf1scoreevaluator_9stoaogs_20250418_020214_869868, log path: /home/codespace/.promptflow/.runs/azure_ai_evaluation_evaluators_f1_score_f1_score_asyncf1scoreevaluator_9stoaogs_20250418_020214_869868/logs.txt


2025-04-18 02:02:12 +0000   26121 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-04-18 02:02:13 +0000   26121 execution.bulk     INFO     Finished 1 / 8 lines.
2025-04-18 02:02:13 +0000   26121 execution.bulk     INFO     Average execution time for completed lines: 1.1 seconds. Estimated time for incomplete lines: 7.7 seconds.
2025-04-18 02:02:13 +0000   26121 execution.bulk     INFO     Finished 2 / 8 lines.
2025-04-18 02:02:13 +0000   26121 execution.bulk     INFO     Average execution time for completed lines: 0.57 seconds. Estimated time for incomplete lines: 3.42 seconds.
2025-04-18 02:02:13 +0000   26121 execution.bulk     INFO     Finished 3 / 8 lines.
2025-04-18 02:02:13 +0000   26121 execution.bulk     INFO     Average execution time for completed lines: 0.39 seconds. Estimated time for incomplete lines: 1.95 seconds.
2025-04-18 02:02:14 +0000   26121 execution.bulk     INFO     Finished 4 / 8 lines.
2025-04

[2025-04-18 02:02:15 +0000][promptflow][WARNING] - Found existing /tmp/azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase__ut4jyi3/flow.flex.yaml, will not respect it in runtime.
[2025-04-18 02:02:15 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-04-18 02:02:15 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_rouge_rouge_asyncrougescoreevaluator_wh3ce_or_20250418_020215_938063, log path: /home/codespace/.promptflow/.runs/azure_ai_evaluation_evaluators_rouge_rouge_asyncrougescoreevaluator_wh3ce_or_20250418_020215_938063/logs.txt


...

[2025-04-18 02:02:19 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-04-18 02:02:19 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-04-18 02:02:19 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_6kolyi7c_20250418_020218_792292, log path: /home/codespace/.promptflow/.runs/azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_6kolyi7c_20250418_020218_792292/logs.txt
[2025-04-18 02:02:19 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_ztgz_12_20250418_020218_830261, log path: /home/codespace/.promptflow/.runs/azure_ai_evaluati

2025-04-18 02:02:17 +0000   26121 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-04-18 02:02:18 +0000   26121 execution.bulk     INFO     Finished 8 / 8 lines.
2025-04-18 02:02:18 +0000   26121 execution.bulk     INFO     Average execution time for completed lines: 0.14 seconds. Estimated time for incomplete lines: 0.0 seconds.
======= Run Summary =======

Run name: "azure_ai_evaluation_evaluators_gleu_gleu_asyncgleuscoreevaluator_q1vy8ddi_20250418_020217_063435"
Run status: "Completed"
Start time: "2025-04-18 02:02:17.053119+00:00"
Duration: "0:00:01.962537"
Output path: "/home/codespace/.promptflow/.runs/azure_ai_evaluation_evaluators_gleu_gleu_asyncgleuscoreevaluator_q1vy8ddi_20250418_020217_063435"

2025-04-18 02:02:28 +0000   26121 execution.bulk     INFO     Finished 8 / 8 lines.
2025-04-18 02:02:28 +0000   26121 execution.bulk     INFO     Average execution time for completed lines: 1.47 seconds. Estimated tim

{'rows': [{'inputs.query': 'What does the Aqua satellite image reveal about the Amazon rainforest in Brazil and Bolivia?', 'inputs.response': 'The Aqua satellite image reveals the following about the Amazon rainforest in Brazil and Bolivia:\n\n- The image was taken in June 2014 in the early afternoon.\n- It shows cumulus clouds forming over the Amazon rainforest.\n- These clouds are a result of water vapor rising on columns of heated air as sunlight warms the forest.\n- The humid air condenses into cumulus clouds when it hits a cooler, more stable air mass above.\n- The cumulus clouds arrange themselves in "cloud streets" which align parallel to the direction of the wind, sometimes showing the direction and intensity of wind.\n- In this particular instance, the clouds line up along the curved lines of high-pressure weather systems.\n\n(Source: page-11.pdf)', 'inputs.context': 'TITLE: page-11.pdf, CONTENT: A\nT\n\nM\nO\n\nS\nP\n\nH\nE\n\nR\nE\n\nE\nA\n\nR\nT\n\nH\n\n4\n\nCurving Cloud S

{"score": 2, "explanation": "The response is mostly unfriendly, as it comes across as judgmental and dismissive."}


{"score": 2, "explanation": "The response is mostly unfriendly, as it comes across as judgmental and dismissive of the other person's choices."}


## View Evaluation Results

Go to your project in Azure AI Foundry and view the results under the Evaluation tab

In [ ]:
print(result['studio_url'])

## Evaluate Using a Custom Evaluator

In [2]:
query="I have a problem with my computer"
response="What? why you spend so much time on that thing? You should be doing something else"

In [3]:
from promptflow.client import load_flow

friendliness_eval = load_flow(source="friendliness.prompty", model={"configuration": model_config})
friendliness_score = friendliness_eval(
    query=query,
    response=response
)
print(friendliness_score)

{"score": 1, "explanation": "The response is unfriendly and dismissive, showing a lack of understanding and support."}


In [8]:
query="I have a problem with my computer"
response="What the f**k? why you spend so much time on that thing? You should be doing something else!! Grrrr"

In [9]:
from promptflow.client import load_flow

friendliness_eval = load_flow(source="friendliness.prompty", model={"configuration": model_config})
friendliness_score = friendliness_eval(
    query=query,
    response=response
)
print(friendliness_score)

{"score": 1, "explanation": "The response is hostile, uses inappropriate language, and is not friendly or constructive."}


In [6]:
query="I have a problem with my computer"
response="I am sorry to hear that you are having a problem with your computer. Can you please provide more details about the issue? I will do my best to help you resolve it."

In [7]:
from promptflow.client import load_flow

friendliness_eval = load_flow(source="friendliness.prompty", model={"configuration": model_config})
friendliness_score = friendliness_eval(
    query=query,
    response=response
)
print(friendliness_score)

{"score": 5, "explanation": "The response is very friendly, empathetic, and shows a willingness to help resolve the issue."}
